<a href="https://colab.research.google.com/github/gabrielcerono/GlioblastomaMultiforme/blob/main/shieh2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk

paper : https://doi.org/10.1371/journal.pone.0233188



 " IDH and MGMT markers were not mandatory for routine pathologic reports " Sadly we won't have this markers in this one, but we have it in other datasets.

In [2]:
dataset = pd.read_excel('/content/shieh.xlsx')

In [3]:
dataset.head(5)

,gender,age,statusPFS,statusOS,surgery,dose,TMZ,Chemogroup,Chemo,diagnostic year,OS,PFS,volume,gender.1,age.1
0,1,63,1,1,1,6000,1.0,1,1,2,1.36,1.260274,731.0,1,63
1,1,64,1,1,0,6600,0.0,2,1,1,8.84,5.115068,NaN,1,64
2,1,62,1,1,1,6000,1.0,1,1,2,1.46,0.641096,215.0,1,62
3,1,71,1,1,1,6000,NaN,3,1,1,2.42,0.652055,255.0,1,71
4,1,21,1,1,0,5940,0.0,2,1,2,1.26,1.079452,154.0,1,21


In [5]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

Let's do all the pre-processing needed

In [6]:
imp_mean = IterativeImputer(random_state=0)

In [65]:
data = imp_mean.fit_transform(dataset)

In [66]:
data = pd.DataFrame(data=data, columns= dataset.columns,)

In [67]:
data.head()

,gender,age,statusPFS,statusOS,surgery,dose,TMZ,Chemogroup,Chemo,diagnostic year,OS,PFS,volume,gender.1,age.1
0,1.0,63.0,1.0,1.0,1.0,6000.0,1.0,1.0,1.0,2.0,1.36,1.260274,731.000000,1.0,63.0
1,1.0,64.0,1.0,1.0,0.0,6600.0,0.0,2.0,1.0,1.0,8.84,5.115068,272.938289,1.0,64.0
2,1.0,62.0,1.0,1.0,1.0,6000.0,1.0,1.0,1.0,2.0,1.46,0.641096,215.000000,1.0,62.0
3,1.0,71.0,1.0,1.0,1.0,6000.0,-1.0,3.0,1.0,1.0,2.42,0.652055,255.000000,1.0,71.0
4,1.0,21.0,1.0,1.0,0.0,5940.0,0.0,2.0,1.0,2.0,1.26,1.079452,154.000000,1.0,21.0


In [68]:
data.drop(columns = ['TMZ', 'gender.1', 'age.1', 'statusPFS', 'PFS'], axis = 1, inplace= True)

In [69]:
data.corr(method = 'pearson')

,gender,age,statusOS,surgery,dose,Chemogroup,Chemo,diagnostic year,OS,volume
gender,1.000000,-0.163487,-0.113228,0.169842,0.117790,-0.153072,-0.030024,-0.037493,0.043419,-0.073217
age,-0.163487,1.000000,0.075994,-0.005687,-0.199397,0.037592,0.110030,0.089632,-0.188313,0.153293
statusOS,-0.113228,0.075994,1.000000,-0.111111,0.026233,0.079058,-0.078567,-0.132453,-0.028040,-0.083594
surgery,0.169842,-0.005687,-0.111111,1.000000,-0.033926,0.020329,0.235702,0.132453,-0.118900,0.088748
dose,0.117790,-0.199397,0.026233,-0.033926,1.000000,0.315258,0.297829,-0.334150,0.468474,-0.162929
Chemogroup,-0.153072,0.037592,0.079058,0.020329,0.315258,1.000000,0.570208,-0.370692,0.002759,0.013387
Chemo,-0.030024,0.110030,-0.078567,0.235702,0.297829,0.570208,1.000000,0.083252,0.115345,-0.049697
diagnostic year,-0.037493,0.089632,-0.132453,0.132453,-0.334150,-0.370692,0.083252,1.000000,-0.345445,-0.077207
OS,0.043419,-0.188313,-0.028040,-0.118900,0.468474,0.002759,0.115345,-0.345445,1.000000,0.127147
volume,-0.073217,0.153293,-0.083594,0.088748,-0.162929,0.013387,-0.049697,-0.077207,0.127147,1.000000


Converting Y to the ndarray so the sksurv package can process it

In [73]:
Y = data[['statusOS', 'OS']]

In [74]:
X = data.drop(columns=['statusOS', 'OS'])

In [75]:
Y['statusOS'] = Y['statusOS'].map({1.0: True, 0: False})


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [76]:
Y = Y.to_records(index=False)

In [77]:
Y

rec.array([( True, 1.36), ( True, 8.84), ( True, 1.46), ( True, 2.42),
           ( True, 1.26), ( True, 0.85), ( True, 6.43), ( True, 0.63),
           ( True, 0.96), ( True, 1.32), ( True, 2.82), ( True, 0.77),
           ( True, 0.84), (False, 2.38), ( True, 0.23), ( True, 1.  ),
           ( True, 3.5 ), ( True, 1.18), ( True, 5.4 ), ( True, 1.93),
           ( True, 0.59), ( True, 0.48), ( True, 2.25), ( True, 1.34),
           ( True, 0.9 ), ( True, 1.12), ( True, 0.58), ( True, 1.01),
           ( True, 0.88), ( True, 0.62), ( True, 1.59), ( True, 2.33),
           ( True, 1.66), ( True, 1.11), ( True, 0.79), ( True, 1.11),
           ( True, 2.92), ( True, 1.22), ( True, 1.9 ), ( True, 2.5 ),
           ( True, 0.32), ( True, 2.22), ( True, 9.15), ( True, 0.45),
           ( True, 1.37), ( True, 3.45), ( True, 5.07), ( True, 1.82),
           ( True, 0.41), ( True, 0.95), ( True, 0.98), ( True, 1.38),
           ( True, 1.82), ( True, 2.68), ( True, 0.89), ( True, 1.68),
      

# Let's do the ranking

In [ ]:
!pip install --upgrade pip
!pip uninstall --yes --quiet osqp
!pip install -U scikit-survival

In [ ]:
pip install eli5

In [ ]:
from sksurv.ensemble import RandomSurvivalForest
from sklearn.model_selection import train_test_split
from eli5.sklearn import PermutationImportance
import eli5


In [78]:
rsf = RandomSurvivalForest()


In [59]:
def permutation(X, y, model, loops):
  rankings = np.zeros(len(X.columns),)
  c_score_total = []

  for x in range(loops):
    
  #Let's do a splitting
    X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.25)
  #Let's train the model on the 
    model.fit(X_train, y_train)
  #Let's define the permutation.
    permuter = PermutationImportance(
    estimator = model,
    n_iter = 10)
  #Let's fit the permutator on the Y test, (This permutation only shuffle the Y test.)
    permuter.fit(X_test, y_test)
  
    columns = X_test.columns.to_list()
  #The feature importance will give us and n array of how much 
    feature_importance = permuter.feature_importances_
    rankings = np.add(feature_importance, rankings)
  #Let's compute the scores
    c_score = model.score(X_test, y_test)
    c_score_total.append(c_score)

  
    
  #Dividing to get the average 
  ranking_terminal = np.true_divide(rankings, loops) 
  c_rank = pd.Series(ranking_terminal, columns).sort_values(ascending = False)
  c_mean = np.mean(np.array(c_score_total))
  return c_rank, c_mean

In [79]:
rank, c_mean = permutation(X, Y, rsf, 100)

In [80]:
rank

age                0.075392
dose               0.038387
diagnostic year    0.022683
gender             0.002137
Chemogroup         0.001975
Chemo              0.000922
surgery           -0.003923
volume            -0.007399
dtype: float64

In [81]:
c_mean

0.6354058571811908